In [1]:
import tensorflow as tf

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import h5py
import cv2
import time
import video_recognition_library as vrl
import video_cnn_recognition_library as cnn

In [5]:
from keras.layers import Activation, Conv3D, Conv2D, Conv1D
from keras.layers import BatchNormalization
from keras.layers import Reshape, Dense, LSTM, GRU, RNN
from keras.models import Model, load_model, Sequential
from keras.layers import Input, add
from keras.layers import MaxPooling3D, GlobalMaxPooling1D, Flatten, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Dropout

In [6]:
def get_few_frames(tensor, center, n_points, step):
    """From video tensor with shape (n_frames, ...) get n_points
    frames around center with some step"""
    indexes = center + np.arange(-n_points//2, n_points//2+1)*step
    indexes = indexes[:tensor.shape[0]]
    tensor = tensor[indexes]
    return tensor

def process_for_frames(tensor):
    tensor = tensor[:,:]
    tensor = get_few_frames(tensor, center=tensor.shape[0]//2, n_points=14, step=5)
    return tensor

In [7]:
base_path = 'bin\\'

In [9]:
metadata = pd.read_csv(base_path + 'metadata.csv')
metadata = metadata.loc[metadata.modality == 'video']
print(len(metadata))

4


In [10]:
def generator_few_frames(batch_size=16, metadata=metadata):
    return vrl.generator_few_frames(batch_size, metadata, base_path, "data_dlib.h5", "labels_dlib.h5", process_for_frames)

In [13]:
g = generator_few_frames()
print(g)

<generator object generator_few_frames at 0x00000275032ED830>


In [14]:
x, label = next(g)

In [16]:
print(x[1])
print(label[1])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[  0   0   0   0   0   0 173 173]


In [17]:
x.shape

(4, 15, 4624)

In [18]:
def model_1(input_shape=(15, 4624), num_classes=8, l2_regularization=0.0001):
    regularization = l2(l2_regularization)
#Result - 0.5
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    model.add(Dense(512, input_shape=input_shape, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [19]:
def model_2(input_shape=(15, 4624), num_classes=8, l2_regularization=0.0001):
    regularization = l2(l2_regularization)
#Result - 0.6
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    model.add(Dense(512, input_shape=input_shape, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [20]:
def model_3(input_shape=(8, 4624), num_classes=8, l2_regularization=0.0001):
    regularization = l2(l2_regularization)
#Result - 0.6
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    model.add(Dense(1024, input_shape=input_shape, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [21]:
def model_4(input_shape=(5, 4624), num_classes=8, l2_regularization=0.01):
    regularization = l2(l2_regularization)
#Result - 0.5
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    model.add(Dense(1024, input_shape=input_shape, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(GRU(256, activation='relu'))
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [22]:
def model_5(input_shape=(8, 4624), num_classes=8, l2_regularization=0.01):
    regularization = l2(l2_regularization)
#Result - 0.65
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    model.add(Conv1D(4, (2), input_shape=input_shape,
               strides=(1), 
               kernel_regularizer=regularization,
               use_bias=False, 
               padding='same'))
    model.add(MaxPooling1D((2), 
                     strides=(1), 
                     padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(GRU(256, activation='relu'))
    #model.add(GlobalMaxPooling1D())
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [23]:
def model_6(input_shape=(15, 4624), num_classes=8, l2_regularization=0.0001):
    regularization = l2(l2_regularization)
#Result - 0.75, 0.58
   
    # base
    img_input = Input(input_shape)
    x = img_input
    
    # create model
    model = Sequential()
    
    model.add(Conv1D(4, (2), input_shape=input_shape,
               strides=(1), 
               kernel_regularizer=regularization,
               use_bias=True, 
               padding='same'))
    model.add(MaxPooling1D((2), 
                     strides=(1), 
                     padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1024, activation='relu'))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(output_dim=8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [24]:
model = model_2()

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15, 512)           2368000   
_________________________________________________________________
dense_2 (Dense)              (None, 15, 512)           262656    
_________________________________________________________________
dense_3 (Dense)              (None, 15, 512)           262656    
_________________________________________________________________
dense_4 (Dense)              (None, 15, 512)           262656    
_________________________________________________________________
dense_5 (Dense)              (None, 15, 512)           262656    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 4104      
Total para

In [27]:
model_name = 'model_11_25_2093_dlib_2'
file_path = model_name + '.hdf5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor='val_loss', mode='min', patience=20)
tensor_board = TensorBoard(log_dir='./logs/' + model_name, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
callbacks_list = [checkpoint, early, tensor_board] 

In [28]:
batch_size = 16
epochs = 20

In [29]:
metadata = metadata.sample(frac=1, random_state=42)
len(metadata)

4

In [31]:
train = metadata.iloc[:int(0.8*metadata.shape[0])]
test = metadata.iloc[int(0.8*metadata.shape[0]):]
len(train)

3

In [32]:
gen_train = generator_few_frames(batch_size, train)
gen_test = generator_few_frames(batch_size, test)

In [33]:
validation_steps=test.shape[0]//batch_size
print(validation_steps)
print(gen_train)

0
<generator object generator_few_frames at 0x00000275055EA888>


In [34]:
print(train)

   actor  emotion emotional_intensity  \
1      1    angry              strong   
3      1    angry              strong   
0      6  fearful              strong   

                                           file_name modality vocal_channel  \
1  ..\DataSet\Video_Song_Actor_01\Actor_01\01-02-...    video          song   
3  ..\DataSet\Video_Song_Actor_01\Actor_01\01-02-...    video          song   
0  ..\DataSet\Video_Speech_Actor_06\Actor_06\02-0...    video        speech   

   angry  calm  disgust  fearful  happy  neutral  sad  surprised  size  \
1      1     0        0        0      0        0    0          0   173   
3      1     0        0        0      0        0    0          0   148   
0      0     0        0        1      0        0    0          0   107   

   size_dlib  
1        173  
3        148  
0        107  


In [31]:
model.fit_generator(gen_train, 
                    epochs=epochs, 
                    steps_per_epoch=train.shape[0]//batch_size, 
                    validation_data=gen_test, 
                    callbacks=callbacks_list,
                    validation_steps=test.shape[0]//batch_size
                   )

Epoch 1/20
29/29 [==============================] - 26s 881ms/step - loss: 2.0694 - acc: 0.1740 - val_loss: 1.9233 - val_acc: 0.2804

Epoch 00001: val_loss improved from inf to 1.92330, saving model to model_11_25_2093_dlib_2.hdf5
Epoch 2/20
29/29 [==============================] - 22s 760ms/step - loss: 1.8366 - acc: 0.2717 - val_loss: 1.9408 - val_acc: 0.2410

Epoch 00002: val_loss did not improve from 1.92330
Epoch 3/20
29/29 [==============================] - 22s 765ms/step - loss: 1.7507 - acc: 0.3123 - val_loss: 1.7296 - val_acc: 0.3043

Epoch 00003: val_loss improved from 1.92330 to 1.72961, saving model to model_11_25_2093_dlib_2.hdf5
Epoch 4/20
29/29 [==============================] - 22s 773ms/step - loss: 1.6441 - acc: 0.3441 - val_loss: 1.5359 - val_acc: 0.4301

Epoch 00004: val_loss improved from 1.72961 to 1.53585, saving model to model_11_25_2093_dlib_2.hdf5
Epoch 5/20
29/29 [==============================] - 22s 760ms/step - loss: 1.5798 - acc: 0.3704 - val_loss: 1.5913